# Azure OpenAI Service - Chat on private data using LangChain

Firstly, create a file called `.env` in this folder, and add the following content, obviously with your values:

```
OPENAI_API_KEY=xxxxxx
OPENAI_API_BASE=https://xxxxxxx.openai.azure.com/
```

Then, let's install all dependencies:

In [1]:
!pip3 install langchain

  Obtaining dependency information for langchain from https://files.pythonhosted.org/packages/e0/2c/1c5e358f954ca23cd20c9220439450a601436ca054a28860e3e1753e64ec/langchain-0.0.312-py3-none-any.whl.metadata
  Obtaining dependency information for SQLAlchemy<3,>=1.4 from https://files.pythonhosted.org/packages/f2/c4/37bb937548c627efbed34ebb629e6fd7c62700227fdc9e38f93c48a4e376/SQLAlchemy-2.0.21-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for anyio<4.0 from https://files.pythonhosted.org/packages/19/24/44299477fe7dcc9cb58d0a57d5a7588d6af2ff403fdd2d47a246c91a3246/anyio-3.7.1-py3-none-any.whl.metadata
  Obtaining dependency information for dataclasses-json<0.7,>=0.5.7 from https://files.pythonhosted.org/packages/21/1f/1cff009cff64420572b9f75b70e4a054095719179a172297dfdd65843162/dataclasses_json-0.6.1-py3-none-any.whl.metadata
  Obtaining dependency information for jsonpatch<2.0,>=1.33 from https://files.pythonhosted.org/packages/73/07/02

In [2]:
import os
import openai
from dotenv import load_dotenv
from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
import numpy as np

# Load environment variables (set OPENAI_API_KEY and OPENAI_API_BASE in .env)
load_dotenv()

# Configure Azure OpenAI Service API
openai.api_type = "azure"
openai.api_version = "2023-03-15-preview"
openai.api_base = os.getenv('OPENAI_API_BASE')
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_type = "azure"

# Init LLM and embeddings model
llm = AzureChatOpenAI(deployment_name="gpt-35-turbo", temperature=0, openai_api_version="2023-03-15-preview")
embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002", 
    openai_api_base=os.getenv('OPENAI_API_BASE'),
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    openai_api_version="2023-03-15-preview",
    openai_api_type="azure",
    chunk_size=1)

embed_vals = embeddings.embed_documents(["Hello, how are you?"])
print(f"got embeddings with shape {np.array(embed_vals).shape}")

got embeddings with shape (1, 1536)


First, we load up our documents from the `data` directory:

In [3]:
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader
from langchain.text_splitter import TokenTextSplitter
from timeit import default_timer

start_time = default_timer()
loader = DirectoryLoader('../data/qna/', glob="*.txt", loader_cls=TextLoader)

documents = loader.load()
text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)
runtime = default_timer() - start_time
print(f"Number of documents: {len(docs)} in {runtime:.2f} seconds")
print(docs)

Number of documents: 6 in 2.69 seconds
[Document(page_content='\n# What is Azure OpenAI?\n\nThe Azure OpenAI service provides REST API access to OpenAI\'s powerful language models including the GPT-3, Codex and Embeddings model series. These models can be easily adapted to your specific task including but not limited to content generation, summarization, semantic search, and natural language to code translation. Users can access the service through REST APIs, Python SDK, or our web-based interface in the Azure OpenAI Studio.\n\n### Features overview\n\n| Feature | Azure OpenAI |\n| --- | --- |\n| Models available | GPT-3 base series <br> Codex series <br> Embeddings series <br> Learn more in our [Models](./concepts/models.md) page.|\n| Fine-tuning | Ada <br> Babbage <br> Curie <br> Cushman* <br> Davinci* <br> \\* available by request. Please open a support request|\n| Price | [Available here](https://azure.microsoft.com/pricing/details/cognitive-services/openai-service/) |\n| Virtual n

Next, let's ingest them into FAISS so we can efficiently query our embeddings:

In [12]:
from langchain.vectorstores import FAISS
db = FAISS.from_documents(documents=docs, embedding=embeddings)

for m in dir(db):
    if not m.startswith("_"):
        print(m)

aadd_documents
aadd_texts
add_documents
add_embeddings
add_texts
adelete
afrom_documents
afrom_texts
amax_marginal_relevance_search
amax_marginal_relevance_search_by_vector
as_retriever
asearch
asimilarity_search
asimilarity_search_by_vector
asimilarity_search_with_relevance_scores
delete
deserialize_from_bytes
distance_strategy
docstore
embedding_function
embeddings
from_documents
from_embeddings
from_texts
index
index_to_docstore_id
load_local
max_marginal_relevance_search
max_marginal_relevance_search_by_vector
max_marginal_relevance_search_with_score_by_vector
merge_from
override_relevance_score_fn
save_local
search
serialize_to_bytes
similarity_search
similarity_search_by_vector
similarity_search_with_relevance_scores
similarity_search_with_score
similarity_search_with_score_by_vector


Now let's create a chain that can do the whole chat on our embedding database:

In [14]:
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate

# Adapt if needed
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template("""Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:""")

qa = ConversationalRetrievalChain.from_llm(llm=llm,
                                           retriever=db.as_retriever(),
                                           condense_question_prompt=CONDENSE_QUESTION_PROMPT,
                                           return_source_documents=True,
                                           verbose=False)
print(qa)

combine_docs_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=ChatPromptTemplate(input_variables=['context', 'question'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template="Use the following pieces of context to answer the users question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='{question}'))]), llm=AzureChatOpenAI(client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, temperature=0.0, openai_api_key='8158d342aa794004ae7a016cb6e8033a', openai_api_base='https://handsonoctober.openai.azure.com', deployment_name='gpt-35-turbo', openai_api_type='azure', openai_api_version='2023-03-15-preview')), document_variable_name='context') question_generator=LLMChain(prompt=PromptTemplate(input_variables=['chat_history', 'question'], template='Given the following 

Now let's ask a question:

In [16]:
chat_history = []
query = "what is azure openai service?"

#This call injects the chat history into the prompt template and then 
#calls the LLM to generate a response to the question and the chat history.
result = qa({"question": query, "chat_history": chat_history})  
print(result["answer"])

Azure OpenAI is a service that provides REST API access to OpenAI's language models, including GPT-3, Codex, and Embeddings model series. These models can be used for content generation, summarization, semantic search, and natural language to code translation. Users can access the service through REST APIs, Python SDK, or a web-based interface in the Azure OpenAI Studio. The service is built on Microsoft Azure and offers features such as virtual network support, managed identity, and responsible AI content filtering. Access to the service is currently limited, and all solutions using the Azure OpenAI service are required to go through a use case review before they can be released for production use.


We can use this to easy implement chat conversations:

In [17]:
chat_history = []

query = "what is Azure OpenAI Service?"
result = qa({"question": query, "chat_history": chat_history})
print("Question:", query)
print("Answer:", result["answer"])

chat_history = #FIXME: Add chat history
query = "Which regions does the service support?"
result = qa({"question": query, "chat_history": chat_history})
print("Question:", query)
print("Answer:", result["answer"])

Question: what is Azure OpenAI Service?
Answer: Azure OpenAI Service is a cloud-based service that provides REST API access to OpenAI's language models, including GPT-3, Codex, and Embeddings model series. These models can be used for various tasks such as content generation, summarization, semantic search, and natural language to code translation. Users can access the service through REST APIs, Python SDK, or a web-based interface in the Azure OpenAI Studio. The service is built on Microsoft Azure and offers features such as virtual network support, managed identity, and responsible AI content filtering. However, access to the service is currently limited due to high demand and Microsoft's commitment to responsible AI.
Question: Which regions does the service support?
Answer: The Azure OpenAI Service is currently available in the East US, South Central US, and West Europe regions.
